<a href="https://colab.research.google.com/github/jvataidee/DataAnalysis/blob/master/Analise_Financeira_Mercado_Gado_de_Corte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**by: [João Ataíde](https://www.joaoataide.com)**
# Análise Mercado Financeira de Gado de Corte


---


## Importanto bibliotecas básicas

In [90]:
#importar bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from statsmodels.tsa.seasonal import seasonal_decompose

# configurar a visualização
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Obtenção dos dados

In [91]:
!pip install investpy -q

In [92]:
!pip install yfinance -q

In [93]:
#importar bibliotecas financeiras
import investpy
import yfinance as yf

### Dados de Commodities

Para fazer as análises do mercado de pecuária, diversas commodities são variáveis para o trabalho, sendo elas as seguintes:



*   `Live Cattle` - preço do Boi Gordo
*   `Feeder Cattle` - preço do Boi em engorda
*   `US Soybeans` - preço da soja
*   `US Corn` - preço do milho
*   `Dollar` - preço do Dollar/Real

Para importar esses dados usarei a biblioteca investpy, que possui um API com comandos para diversos dados financeiros. Para isso, precisaremos estipular uma janela de programação no qual precisa ser maior que 6 anos, devido ao ciclo da pecuária de corte.



In [94]:
#Lista de commodities
commodities = ["Live Cattle", "Feeder Cattle", "US Soybeans","US Corn"]

#Data de inicio e fim de análise
inicio = "01/01/2004"
fim = "31/12/2021"

In [ ]:
#Função para importar dados das commodities
def commodities_df(commodities, inicio,fim):
  precos = pd.DataFrame()
  for i in commodities:
    df = investpy.commodities.get_commodity_historical_data(commodity=i, from_date=inicio, to_date=fim).Close
    df.rename(i, inplace=True)
    df.columns = [i]
    precos = pd.concat([precos,df], axis=1)
    precos.index.name="Date"
  return precos


#Data Frame de commodites
df = commodities_df(commodities,inicio,fim)

#Importar dollar dos anos de 2004 e 2020
doll = yf.Ticker("USDBRL=x")
dollar = pd.DataFrame(doll.history(start = "2004-01-01", end ="2020-12-31" ).Close)
dollar.columns = ["Dollar"]
dollar.index.name="Date"

#merge dado
dados =pd.merge(df, dollar, how='inner', right_index=True, left_index=True)

#Agrupar a média por trimestre
dados_trimestre = dados.groupby(pd.Grouper(freq='Q')).mean()
dados_trimestre = dados_trimestre.reset_index()
dados_trimestre.head()

In [ ]:
dados_trimestre.shape

### Dados do SIDRA

Outro dados importante é referente aos número de abates dos animais por classe dentro do intervalo de tempo, sendo usado então validar os ciclos e as sazonalidades das culturas. Para isso, precisaremos baixar todas as classes os dados referentes ao intervalo e cabeças abatidas de cada classe:

*   `Bois` - Animal macho
*   `Vacas` - Animal fêmea
*   `Novilhas` - Animal fêmea mais nova 
*   `Novilhos` - Animal macho mais novo
*   `Totais` -  Soma de todas as classes



In [ ]:
#link do banco de dados sidra
consulta = 'https://apisidra.ibge.gov.br/values/t/1092/n1/all/v/all/p/all/c12716/all/c18/55,56,992,111734,111735/c12529/all/d/v1000151%202,v1000284%202,v1000285%202'
ufs = [5] #numero de consultas

#Baixar dados do sidra
df2 = pd.DataFrame()
for uf in ufs:
    url = consulta.format(uf)
    dados_json = requests.get(url).json()
    dados_pandas = pd.DataFrame(dados_json)
    df2 = pd.concat([dados_pandas,df2])

In [ ]:
#Renomear colunas
sidra = df2.copy() 
colunas = list(df2.iloc[0,:])
sidra.columns = colunas

#Filtrar dados
sidra = sidra.iloc[1:,:]
sidra = sidra[sidra["Variável"]=="Animais abatidos"]
sidra = sidra[sidra["Tipo de inspeção"]=="Total"]
sidra = sidra[sidra["Unidade de Medida"]=="Cabeças"]
sidra = sidra[sidra["Referência temporal"]=="Total do trimestre"]

#Escolher colunas necessárias para a análise
sidra = sidra.reset_index()
sidra = sidra.iloc[:, [5, 11, 15]]

#Dropar coluna de valor X
sidra = sidra.drop(sidra[sidra["Valor"] == 'X'].index)

#tranformar valor em float
sidra["Valor"] = sidra["Valor"].astype(float)

#Encontrar datas
times = sidra.Trimestre.str.split(expand=True)
times[2] = pd.to_datetime(times[2])
sidra3 = sidra.copy()
sidra3 = sidra3[times[2] >= "2004"]
sidra3 = sidra3[times[2] < "2021"]
sidra3.head()

In [ ]:
#Separar dados pelo tipo de rebanho
bois = sidra3[sidra3["Tipo de rebanho bovino"] == "Bois"]
vacas = sidra3[sidra3["Tipo de rebanho bovino"] == "Vacas"]
novilhas = sidra3[sidra3["Tipo de rebanho bovino"] == "Novilhos"]
novilhos = sidra3[sidra3["Tipo de rebanho bovino"] == "Novilhas"]
total = sidra3[sidra3["Tipo de rebanho bovino"] == "Total"]

In [ ]:
#Função de pré join
def pre_join(dataframe, nome):
  dataframe = dataframe.reset_index()
  dataframe = dataframe.iloc[:, [1, 2]]
  dataframe.columns = [nome, "Trimestre"]
  return dataframe

#Criar contador de trimestre
bois = pre_join(bois, "Bois")
vacas = pre_join(vacas, "Vacas")
novilhas = pre_join(novilhas, "Novilhas")
novilhos = pre_join(novilhos, "Novilhos")
total = pre_join(total, "Total")

In [ ]:
#Joins de todos os datasets
df_final = dados_trimestre.copy()
df_final = df_final.join(bois)
df_final = df_final.join(vacas.Vacas)
df_final = df_final.join(novilhas.Novilhas)
df_final = df_final.join(novilhos.Novilhos)
df_final = df_final.join(total.Total)

#indexar por data
df_final=df_final.set_index("Date")
df_final.head()

In [ ]:
#Criar coluna de média todos os trimestres 
df_final["Média Bois"] = df_final["Bois"].mean()
df_final["Média Vacas"] = df_final["Vacas"].mean()
df_final["Média Novilhas"] = df_final["Novilhas"].mean()
df_final["Média Novilhos"] = df_final["Novilhos"].mean()
df_final["Média Total"] = df_final["Total"].mean()

## Sazonalidade

A sazonalidade são eventos que costumam ocorrer em determinado momento do ano, assim como toda e qualquer cultura a pecuária de corte não é diferente, entretanto, tal se diferência devido ao seu comportameto plurianual, modificando então suas cultura a decorrer da passagem de alguns anos, que geralmente são de 5 à 6 anos.

In [ ]:
#Agrupar por trimestre
trimestre = df_final.Trimestre.str.split(expand=True)
trimestre = df_final.join(trimestre)
trimestre = trimestre.groupby([0,2]).sum()
trimestre= trimestre.reset_index()
trimestre.head()

In [ ]:
#Dataset por ano
df_2014 = trimestre[trimestre[2]== "2014"]
df_2015 = trimestre[trimestre[2]== "2015"]
df_2016 = trimestre[trimestre[2]== "2016"]
df_2017 = trimestre[trimestre[2]== "2017"]
df_2018 = trimestre[trimestre[2]== "2018"]
df_2019 = trimestre[trimestre[2]== "2019"]
df_2020 = trimestre[trimestre[2]== "2020"]

#criar dataset abate de vacas por ano
df_vacas_ano = pd.DataFrame()
df_vacas_ano["2014"]= list(df_2014.Vacas)
df_vacas_ano["2015"] = list(df_2015.Vacas)
df_vacas_ano["2016"]= list(df_2016.Vacas)
df_vacas_ano["2017"]= list(df_2017.Vacas)
df_vacas_ano["2018"]= list(df_2018.Vacas)
df_vacas_ano["2019"]= list(df_2019.Vacas)
df_vacas_ano["2020"]= list(df_2020.Vacas)
df_vacas_ano

In [ ]:
df_vacas_ano.plot(kind="bar", figsize = (12,4));

Eu agrupei o número de abates das fêmeas por ano e trimestre, assim foi possível identificar que no primeiro trimestre é sempre onde tem maior número de abate de fêmeas. Isso mostra a importância da questão da oferta de fêmeas no período de abate, pois quando a oferta está muito alta ela tende a preencher a escala do frigorifico e pressionar o preços da @ de macho.

**Mas por que isso é mais forte no primeiro trimestre?** isso é devido ao processo de classificação do produtor quanto as fêmeas que não irão emprenhar novamente no outro ano, assim decorre o descarte maior dessas no primeiro e no segundo semestre. Outra quesão é a redução que ocorre no terceiro trimestre que é devido a entressafra, decorrente da não existência de bois de pasto e o momento com menor oferta de fêmeas.

### Sazonalidade do Boi Gordo

In [ ]:
#Pré-processando dados
dados2 = df.copy()
boi_gordo = pd.DataFrame(dados2["Live Cattle"].fillna(dados2["Live Cattle"].median()))
boi_gordo = boi_gordo.reset_index()
boi_gordo = boi_gordo.groupby(pd.Grouper(key='Date',freq='M')).first()
boi_gordo2 = boi_gordo[boi_gordo.index > '2014-01-01']
boi_gordo3 = boi_gordo[boi_gordo.index > '2017-01-01']
boi_gordo.head()

In [ ]:
#Calcular decomposição da série
boi_gordo_seasonal = seasonal_decompose(boi_gordo)
boi_gordo2_seasonal = seasonal_decompose(boi_gordo2)
boi_gordo3_seasonal = seasonal_decompose(boi_gordo3)

In [ ]:
# Plotar decomposição e série do boi gordo
fig, ax = plt.subplots(4,figsize = (10,8))
ax[0].plot(boi_gordo_seasonal.observed)
ax[1].plot(boi_gordo_seasonal.trend)
ax[2].plot(boi_gordo_seasonal.seasonal)
ax[3].plot(boi_gordo_seasonal.resid)

ax[0].set_title('Observado')
ax[1].set_title('Tendência')
ax[2].set_title('Sazonalidade')
ax[3].set_title('Ruído')
fig.autofmt_xdate()
plt.show()

Olhando o gráfico acima podemos ver a decomposição da série temporal do preço do boi gordo no intervalo de 2014 a 2019, indicando quais as tendência e sazonalide do preço do boi gordo.

In [ ]:
#escolhendo 1 ano para sazonalidade de 1997 a 2020
boi_plot = pd.DataFrame(boi_gordo_seasonal.seasonal)
boi_plot = boi_plot.reset_index()
boi_plot = boi_plot[boi_plot["Date"] >= "2020-01-01"]
boi_plot = boi_plot[boi_plot["Date"] <= "2020-12-31"]
boi_plot["Média"] = boi_plot["Live Cattle"].mean()

#escolhendo 1 ano para sazonalidade de 2014 a 2020
boi_plot2 = pd.DataFrame(boi_gordo2_seasonal.seasonal)
boi_plot2 = boi_plot2.reset_index()
boi_plot2 = boi_plot2[boi_plot2["Date"] >= "2020-01-01"]
boi_plot2 = boi_plot2[boi_plot2["Date"] <= "2020-12-31"]

#escolhendo 1 ano para sazonalidade de 2017 a 2020
boi_plot3 = pd.DataFrame(boi_gordo3_seasonal.seasonal)
boi_plot3 = boi_plot3.reset_index()
boi_plot3 = boi_plot3[boi_plot3["Date"] >= "2020-01-01"]
boi_plot3 = boi_plot3[boi_plot3["Date"] <= "2020-12-31"]


In [ ]:
#plotar sazonalidade de 1 ano
fig, ax = plt.subplots(figsize = (12,4))

ax.plot(boi_plot["Date"], boi_plot["Live Cattle"],label='1997 - 2020')
ax.plot(boi_plot["Date"], boi_plot["Média"], ls ='--', color= "Grey",label='Média')
ax.plot(boi_plot2["Date"], boi_plot2["Live Cattle"], ls ='--', color= "Green",label='2014 - 2020', alpha = 0.2)
ax.plot(boi_plot3["Date"], boi_plot3["Live Cattle"], ls ='--', color= "Red",label='2017 - 2020', alpha = 0.2)
ax.set_title('Sazonalidades Boi Gordo')
ax.legend()
plt.show()

Dando um zoom para o intervalo de 1 ano, podemos ver que no começo do ano o preço começa maior que a média, assim, tendo uma queda brusca entre o més de máio e junho, subindo novamente entre o més de novembro e dezenbro, onde repete novamente. 

Logicamente devido a pecuária ser uma industria de céu aberto está sujeita a enteperes o que podem variar o preço, além de uma mudança bruca de mercado financeiro relacionado a demanda, assim tal modelo apresentado acima uma estimativa.


Agora olhe a difenrença quando colocamos um invervalo menor para identificar a sazonalidade, veja que o valores de baixa estão aumentando, isso é devido a o maior número de criação por confinamento nos útimos anos, tal efeito se salienta quando incurtamos ainda mais o período de análise, sendo então que a tendência está em mover os menores preço para os meses de setembro e outubro.


### Sazonalidade do Bezerro

In [ ]:
#Pré-processando dados
dados2 = df.copy()
bezerro = pd.DataFrame(dados2["Feeder Cattle"].fillna(dados2["Live Cattle"].median()))
bezerro = bezerro.reset_index()
bezerro = bezerro.groupby(pd.Grouper(key='Date',freq='M')).first()
bezerro.head()

In [ ]:
#Calcular decomposição da série
bezerro_seasonal = seasonal_decompose(bezerro)

In [ ]:
#escolhendo 1 ano
bezerro_plot = pd.DataFrame(bezerro_seasonal.seasonal)
bezerro_plot = bezerro_plot.reset_index()
bezerro_plot = bezerro_plot[bezerro_plot["Date"] >= "2020-01-01"]
bezerro_plot = bezerro_plot[bezerro_plot["Date"] <= "2020-12-31"]
bezerro_plot["Média"] = bezerro_plot["Feeder Cattle"].mean()


In [ ]:
#plotar sazonalidade de 1 ano
fig, ax = plt.subplots(figsize = (12,4))

ax.plot(bezerro_plot["Date"], bezerro_plot["Feeder Cattle"], label='1997 - 2020')
ax.plot(bezerro_plot["Date"], bezerro_plot["Média"], ls ='--', color= "Grey",label='Média')
ax.set_title('Sazonalidades do Bezerro')
ax.legend()
plt.show()

Note que referente a o bezerro, a sazonalidade é bem diferente do anterior, sua alta de preço está por meados de outurbo e novembro, com uma subida entre os mesdes de baixa do boi gordo, além disso, o seu maior preíodo de baixa está no começo do ano, isso se dá pelo peso do animal, pois devido ao tempo de reprodução que ocorre no final do ano os bezerro ainda não estão magro.

## Ciclos

### interferência das fêmas

In [ ]:
#dataset participação das fêmeas no abate total
participacao_femeas = pd.DataFrame()
participacao_femeas["Trimestre"] = df_final['Trimestre']
participacao_femeas["Participacao"] = (df_final["Vacas"]/df_final["Total"])*100
participacao_femeas["Alto"] = participacao_femeas["Participacao"].max()
participacao_femeas["Baixa"] = participacao_femeas["Participacao"].min()
participacao_femeas["Média"] = participacao_femeas["Participacao"].mean()
participacao_femeas["Bezerro"] = df_final["Feeder Cattle"]
participacao_femeas.head()

In [ ]:
#Plot participação das fêmeas
fig, ax = plt.subplots( figsize = (15,6))

ax.bar(participacao_femeas["Trimestre"], participacao_femeas["Participacao"], alpha = 0.5)
ax.plot(participacao_femeas["Trimestre"], participacao_femeas["Participacao"] +10, color = "red", ls = "--")
ax.plot(participacao_femeas["Trimestre"], participacao_femeas["Alto"] +10, color = "grey", ls = "--")
ax.plot(participacao_femeas["Trimestre"], participacao_femeas["Baixa"] +10, color = "grey", ls = "--")
ax.plot(participacao_femeas["Trimestre"], participacao_femeas["Média"] +10, color = "grey", ls = "--")
ax.set_ylabel("% de Participação")
ax.set_title("Participação das fêmeas no abate trimestral total")
ax.set_ylim(20, 55)
ax.set_xticklabels(participacao_femeas["Trimestre"], rotation = 90)
plt.show()

No gráfico acima podemos ver exatamente a participação das fêmeas no abate total, na qual mostra a força que estas possuem no preço do boi gordo. Note que a linha vermelha e a linha cinza é o preço máximo, projeções demonstram da osiosidade do abate do animal, no qual demonstra um ciclo que dura aproximadamente de 5 a 6 anos podendo varia ao longo do tempo.

In [ ]:
#Plot participação das fêmeas
fig, ax1 = plt.subplots( figsize = (12,4))

ax1.bar(participacao_femeas["Trimestre"], participacao_femeas["Participacao"], alpha = 0.5)

ax2 = ax1.twinx()
ax2.plot(participacao_femeas["Trimestre"], participacao_femeas["Bezerro"], color = "red", ls = "--")

ax1.set_ylabel("% de Participação")
ax1.set_title("Participação das fêmeas no abate trimestral total")
ax1.set_ylim(20, 55)
ax1.set_xlim(35, 68)
ax1.set_xticklabels(participacao_femeas["Trimestre"], rotation = 90)

plt.show()